In [ ]:
import pandas as pd
import os
import sys
import json

In [28]:
if sys.platform.startswith("linux"):
    data_dir = "/bask/projects/v/vjgo8416-amber/data/gbif_download_standalone/"
elif sys.platform == "darwin":
    data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"
else:
    print("Not linux or mac!")

checklist_name = "uksi-moths"

df = pd.read_csv(os.path.join("../species_checklists",checklist_name+"-keys-nodup.csv"))

In [ ]:
df["n_imgs"] = ""
for idx, row in df.iterrows():
    
    family  = row["family_name"]
    genus   = row["genus_name"]
    species = row["gbif_species_name"]
    
    # Check if directory exists 
    species_dir = (
        os.path.join(data_dir,"gbif_images","sandbox",family,genus,species)
        )
    
    if os.path.isdir(species_dir):
        n_images_on_disk = len(
            [f for f in os.listdir(species_dir) if f.lower().endswith('.jpg') ]
            )
        # print(n_images_on_disk)
        
        # Load metadata
        try:
            
            with open(os.path.join(species_dir,"meta_data.json")) as file:
            
                meta_data = json.load(file)
            
            # Count the number of images for this species
            n_imgs_metadata = 0

            for key, value in meta_data.items():
                if value.get("image_is_downloaded") == True:
                    n_imgs_metadata += 1
            
            # Do n images match?
            if n_images_on_disk == n_imgs_metadata:
                # print(f"N images match for {species_dir}")
                pass
            else:
                print(f"N images MISmatch for {species_dir}")
        except Exception as e:
            print(f"No metadata for {species_dir}")
            
    else:
        
        n_images_on_disk = 0
    
    # print(f"{species} has {n_images_on_disk} images")
    
    # Record this
    df.loc[idx, "n_imgs"] = n_images_on_disk
    
    

In [29]:
# Save the df
df.to_csv(os.path.join("../data_stats_files/","data_stats_"+checklist_name+".csv"))